<a href="https://colab.research.google.com/github/HardikPaliwal/CS484Proj/blob/master/cs484%20proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CS484 Final Project

#### Topic 6: Weakly supervised classification

#### Hardik Paliwal (20725413), Lance Pereira (20719626)

______________________________________________________________

**Table Of Contents**
- A) Abstract
- B) High Level Goals and Methedology
- C) Team Members and Contributions
- D) External Code Libraries
- E) Experiments
- F) Visual Results
- ?) Conclusions

**A) Abstract:**

For our project we have decided to do Project 6, choosing specifically MNIST and Fashion MNIST with a subset of digits and clothes. 
 

**B) High Level Goals and Methedology:**

Out high level goal is to use weakly supervised classification to improve our 
prediction ability compared to just training using labeled images. We hope to 
achieve atleast a 5% increase in our test prediction score using clustering 
methods.

Our method is split into two experiments, 
- we first will test to see training
a CNN on N-M labeled images, then use a cluestering method to classify all the 
N images, using the majority label in each cluster as the predicted label
- secondly we will try using the predicted labels from the previous step to 
train a new CNN model, to see if it performs better

Our baseline will be a simple CNN of N+1 (with the K extra classes being labelled as "unknown") to differentiate between unlabeled and labeled classes. Then simply run (some unsupervised model) on the unlabeled data. 

**C) Team Members and Contributions**

- Hardik Paliwal
  - Created CNN based on VGG11 for training
  - Created method to get predicted labels from clustering methods

- Lance Pereira
  - Modified VGG11 like CNN for better results
  - Created experiments for ratios of M/N 
  - Created experiments for diffirent clustering methods
    - Kmeans
    - GMM
  - Created experiments for final classification using Kmeans vs using predicted labels to retrain CNN

**D) External Code Libraries**

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision as tv
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.autograd import Variable
import sklearn
from torch import optim
import numpy as np
%matplotlib inline

In [17]:
dev=torch.device("cuda") 

In [3]:
trainset = tv.datasets.MNIST(root="./", download=True,train=True,  transform=tv.transforms.Compose(
    [tv.transforms.Resize(32), tv.transforms.ToTensor()]))
trainloader = DataLoader(trainset, batch_size=128, shuffle=True)

testset = tv.datasets.MNIST(root="./", download=True,train=False,  transform=tv.transforms.Compose(
    [tv.transforms.Resize(32), tv.transforms.ToTensor()]))
testloader = DataLoader(testset, batch_size=128, shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [4]:
#Modifies dataset in place to only have values correspounding to the labels in classesToUse
def extractClasses(dataset, classesToUse):
    idx = dataset.targets == classesToUse[0]
    for k in classesToUse:
        idx = (idx | (dataset.targets ==k))
    dataset.targets = dataset.targets[idx]
    dataset.data = dataset.data[idx]
    return dataset

#modifies dataset to change classesToSet labels to -1 (which we will assume stands for unknown class)
def setClassToUnknown(dataset, classesToSet):
  idx = dataset.targets == classesToSet[0]
  for k in classesToSet:
      idx = (idx | (dataset.targets ==k))
  dataset.targets[idx] = -1
  return dataset

In [5]:
#This is an implementation of VGG11 (which is a precursor to VGG16) for mnist dataset.
# it also takes in n, which is the number of classes. N+1 class stands for unknown. 

#this will let us differeniate the unlabelled data from the labelled data
class BasicNet(nn.Module):
    def __init__(self, n=9):
        super(BasicNet, self).__init__()
        self.batchNorm = [nn.BatchNorm2d(64), nn.BatchNorm2d(128),nn.BatchNorm2d(256), nn.BatchNorm2d(256),
                          nn.BatchNorm2d(512), nn.BatchNorm2d(512), nn.BatchNorm2d(512), nn.BatchNorm2d(512)]
        self.conv = [
        nn.Conv2d(1, 64, 3, 1, 1) ,nn.Conv2d(64, 128, 3, 1, 1), nn.Conv2d(128, 256, 3, 1, 1), nn.Conv2d(256, 256, 3, 1, 1)
       ,nn.Conv2d(256, 512, 3, 1, 1), nn.Conv2d(512, 512, 3, 1, 1), nn.Conv2d(512, 512, 3, 1, 1), nn.Conv2d(512, 512, 3, 1, 1)
        ]
        maxPool = nn.MaxPool2d(2, stride=2)
        self.conv1 = nn.Sequential(self.conv[0], self.batchNorm[0], nn.ReLU(), maxPool)
        self.conv2 = nn.Sequential(self.conv[1], self.batchNorm[1], nn.ReLU(), maxPool)
        self.conv3 = nn.Sequential(self.conv[2], self.batchNorm[2], nn.ReLU())
        self.conv4 = nn.Sequential(self.conv[3], self.batchNorm[3], nn.ReLU(), maxPool) 
        self.conv5 = nn.Sequential(self.conv[4], self.batchNorm[4], nn.ReLU())
        self.conv6 = nn.Sequential(self.conv[5], self.batchNorm[5], nn.ReLU(), maxPool)
        self.conv7 = nn.Sequential(self.conv[6], self.batchNorm[6], nn.ReLU()) 
        self.conv8 = nn.Sequential(self.conv[7], self.batchNorm[7], nn.ReLU(), maxPool)
        self.fc1 = nn.Linear(512, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, n+1)
        
    def forward(self, x, feature_embedding=False):
        dropOut = nn.Dropout(p=0.5)

        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.conv8(x)
        x = torch.flatten(x, 1)
        
        x = dropOut(F.relu(self.fc1(x)))
        x = dropOut(F.relu(self.fc2(x)))
        if(feature_embedding):
          return x
        #Not sure why this works without softmax. probably a reasoning givin in the paper (cause the output can range from anything (not normalized to a 0-1 probability range))
        x = self.fc3(x)
        return x

In [6]:
def test(data, net):
    net.eval()
    loss_func = nn.CrossEntropyLoss()

    total_correct = 0
    total_loss = 0
    with torch.no_grad():
        correct = 0
        total = 0
        for i, (images, labels) in enumerate(data):
            images= images.to(dev)
            labels = labels.to(dev)
            test_pred = net(images)

            pred = torch.max(test_pred, 1)[1].data.squeeze()
            total_correct+= (pred == labels).sum().item()
            loss = loss_func( test_pred, labels)
            total_loss+= loss.item()*images.size(0)
        return total_correct/len(data.dataset), total_loss/len(data.dataset)
  
def train(num_epochs, net, trainloader):
    optimizer = optim.SGD(net.parameters(), lr=0.01)
    loss_func = nn.CrossEntropyLoss()

    accuracy_through_epochs = []
    total_step = len(trainloader)
    
    for epoch in range(num_epochs):
        net.train()
        for i, (images, labels) in enumerate(trainloader):
            images= images.to(dev)
            labels = labels.to(dev)
            optimizer.zero_grad()           
            prediction = net(images)
            loss = loss_func( prediction, labels)
            loss.backward()
            optimizer.step()
            if ((i +1) % 100 == 0):
                print(f"Epoch {epoch+1} / {num_epochs}, Step {i+1}/ {total_step} , Loss {loss.item()}")

    return accuracy_through_epochs, net

In [7]:
net = BasicNet()
net.to(dev)
result, trained_net = train(4, net, trainloader)

Epoch 1 / 4, Step 100/ 469 , Loss 0.2969116270542145
Epoch 1 / 4, Step 200/ 469 , Loss 0.10553447902202606
Epoch 1 / 4, Step 300/ 469 , Loss 0.0899878740310669
Epoch 1 / 4, Step 400/ 469 , Loss 0.05863155424594879
Epoch 2 / 4, Step 100/ 469 , Loss 0.01627485454082489
Epoch 2 / 4, Step 200/ 469 , Loss 0.03881200775504112
Epoch 2 / 4, Step 300/ 469 , Loss 0.012568647973239422
Epoch 2 / 4, Step 400/ 469 , Loss 0.047573961317539215
Epoch 3 / 4, Step 100/ 469 , Loss 0.0049950238317251205
Epoch 3 / 4, Step 200/ 469 , Loss 0.005219787824898958
Epoch 3 / 4, Step 300/ 469 , Loss 0.013199671171605587
Epoch 3 / 4, Step 400/ 469 , Loss 0.009626945480704308
Epoch 4 / 4, Step 100/ 469 , Loss 0.05020418018102646
Epoch 4 / 4, Step 200/ 469 , Loss 0.017388591542840004
Epoch 4 / 4, Step 300/ 469 , Loss 0.0251016765832901
Epoch 4 / 4, Step 400/ 469 , Loss 0.002310581272467971


In [8]:
#do this to store the results in 1 numpy array of 10000 images vs like 20 batches of size 128 images. 
#get memory error when doing it on a batch of size len(trainloader), so we have to combine the results for trainloader
testloaderBig = DataLoader(testset, batch_size=len(testset.targets))
trainloaderBig = DataLoader(trainset, batch_size=len(testset.targets))

featureEmbed = []
predictedDigit = []
testFeatureEmbed = ""
testPredictedDigit = ""
with torch.no_grad():
  for i, (images, labels) in enumerate(trainloaderBig):
      images= images.to(dev)
      labels = labels.to(dev)
      featureEmbed.append(net(images, feature_embedding=True).to("cpu").numpy())
      pred = net(images)
      predictedDigit.append(torch.max(pred, 1)[1].data.squeeze().to("cpu").numpy())

  for i, (images, labels) in enumerate(testloaderBig): #only runs once =D
      images= images.to(dev)
      labels = labels.to(dev)
      testFeatureEmbed= net(images, feature_embedding=True).to("cpu").numpy()
      pred = net(images)
      testPredictedDigit = torch.max(pred, 1)[1].data.squeeze().to("cpu").numpy()

#combine them
tmp = featureEmbed[0]
tmp1 = predictedDigit[0]
for i in range(1, len(featureEmbed)):
  tmp = np.concatenate((tmp, featureEmbed[i]))
  tmp1 = np.concatenate((tmp1, predictedDigit[i]))

# TODO(Hardik) why did you do this line?
featureEmbed = tmp
predictedDigit = tmp1

In [22]:
featureEmbed.shape

(60000, 4096)

In [9]:
from sklearn.cluster import MiniBatchKMeans
#should change this when we actually drop classes. Currently running with all classes. 
n=5
d=5
kmeans = MiniBatchKMeans(n_clusters = n+d+20)
kmeans.fit(featureEmbed)

MiniBatchKMeans(n_clusters=30)

In [20]:
# featureEmbed.shape
testFeatureEmbed.shape

(10000, 4096)

In [11]:
#In order to see how well k-means did we can use this supervised method of defining what a cluster is by seting the cluster label as the most common digits in that cluster
#unsupervised approaches include: manually selecting class depending on mean image
def retrieve_info(cluster_labels,y_train):
  reference_labels = {}
# For loop to run through each label of cluster label
  for i in range(len(np.unique(kmeans.labels_))):
    index = np.where(cluster_labels == i,1,0)
    num = np.bincount(y_train[index==1]).argmax()
    reference_labels[i] = num
  return reference_labels

In [12]:
reference_labels = retrieve_info(kmeans.labels_,trainset.targets.numpy())
number_labels = np.zeros(testFeatureEmbed.shape[0]).astype(int)
predicted_test = kmeans.predict(testFeatureEmbed)
for i in range(testFeatureEmbed.shape[0]):
  number_labels[i] = reference_labels[predicted_test[i]]

In [27]:
reference_labels

{0: 8,
 1: 3,
 2: 2,
 3: 8,
 4: 6,
 5: 0,
 6: 9,
 7: 0,
 8: 3,
 9: 7,
 10: 8,
 11: 0,
 12: 4,
 13: 5,
 14: 3,
 15: 3,
 16: 1,
 17: 4,
 18: 4,
 19: 7,
 20: 0,
 21: 7,
 22: 0,
 23: 6,
 24: 1,
 25: 2,
 26: 1,
 27: 4,
 28: 9,
 29: 8}

In [13]:
#BUT Note our 30 classes for k-means and the use of the supervised cluster classification. We only get around 0.5 with 10 classes.
#also note that I get varying results, from 0.9 to 0.99 when running on 30 clusters.
target_test = testset.targets.numpy()
accuracy = np.sum(np.where(number_labels == target_test, 1, 0)) / target_test.shape[0]
print(f"Our accuracy with kmeans is: {accuracy} on test set from training on trainset" )

Our accuracy with kmeans is: 0.9049 on test set from training on trainset


**Conclusions**

- We found that...